# `Промышленное машинное обучение на Spark`
## `Занятие 05: Feature Engineering`

О чём можно узнать из этого ноутбука:

* Accumulator/Broadcast
* Градиентный спуск
* Винзоризация
* Нормализация данных

In [1]:
! pip3 install pyspark pyarrow

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import numpy as np
import random

import pyspark
from pyspark.sql import Window
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

from pyspark import SparkConf, SparkContext

In [3]:
conf = (
    SparkConf()
        .set('spark.ui.port', '4050')
        .setMaster('local[*]')
)
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

23/11/11 08:21:43 WARN Utils: Your hostname, vm-01 resolves to a loopback address: 127.0.1.1; using 10.128.0.16 instead (on interface eth0)
23/11/11 08:21:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/11 08:21:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/11 08:21:58 WARN Utils: Service 'SparkUI' could not bind on port 4050. Attempting port 4051.
23/11/11 08:21:58 WARN Utils: Service 'SparkUI' could not bind on port 4051. Attempting port 4052.
23/11/11 08:21:58 WARN Utils: Service 'SparkUI' could not bind on port 4052. Attempting port 4053.
23/11/11 08:21:58 WARN Utils: Service 'SparkUI' could not bind on port 4053. Attempting port 4054.
23/11/11 08:21:58 WARN Utils: Service 'SparkUI' could not bind on port 4054. Attempting port 4055.


### `Общие данные в Spark`

При работе в распределённых окружениях нужно учитывать, что разные воркеры имеют прямой доступ только к своим локальным данным. Как следствие, алгоритмы в таких системах должны учитывать отсутствие общей памяти в целом и проблемы синхронизации между отдельными процессами в частности.

Для решения этой проблемы в Spark предложены два средства: Accumulator и Broadcast.

#### `Accumulator`

Аккумуляторы в Spark представляют из себя общую переменную, которые отдельные воркеры могут обновлять, но не могут считывать (так как значение этой переменной однозначно не определено по причине необходимости дорогостоящей синхронизации между отдельными процессами). 

А таже поддерживают единственную операцию: `+=` (inplace add, `.__iadd__`) — коммутативную, ассоциативную операцию сложения. После того, как воркеры перестанут изменять переменную её значение доступно только на spark-драйвере через атрибут `.value`.

Пример использования аккумуляторов — подсчёт общих статистик в процессе вычислений, например, суммарное значение функции потерь (см. пример с GD ниже), общее число слов в датасете и так далее.

In [5]:
# создание такой переменной 
# возможно только через SparkContext
acc = sc.accumulator(value=0)
acc

Accumulator<id=0, value=0>

Через атрибут `.value` посмотрим содержимое данной переменной

In [6]:
acc.value

0

In [7]:
# отправляем на каждый из worker-нод данные
rdd = sc.parallelize([1, 2, 3, -4, 5])
# метод .add тоже самое что и +=
rdd.foreach(lambda x: acc.add(x))
acc.value

7

In [8]:
acc.value = 4
acc.value

4

В предыдущем примере для работы с accumulator использовалась анонимная lambda-функция. Но можно пользоваться и обычными именованными функциями с доступом к переменной через объявление её global.

In [9]:
acc_sum = sc.accumulator(0)

def count(x):
    global acc_sum
    acc_sum += x

# теперь вместо lambda-функции указываем 
# имя созданной выше
rdd.foreach(count)
acc_sum.value

7

In [10]:
acc_cnt = sc.accumulator(0)
rdd_02 = sc.parallelize([1, 2, 3, 4, 5, 6, 7, 8])

rdd.foreach(lambda x: acc_cnt.add(1))
rdd_02.foreach(lambda x: acc_cnt.add(1))
acc_cnt.value

13

В spark можно определять аккумуляторы собственного типа. Для этого необходимо создать класс-наследник класса AccumulatorParam. И реализовать в нём два метода:
- **zero** - "нулевое" значение для созданного нового типа аккумулятора
- **addInPlace** - описание логики сложения

In [11]:
from pyspark.accumulators import AccumulatorParam

class VectorAccumulatorParam(AccumulatorParam):
    def zero(self, value):
        return [0.0] * len(value)
    
    def addInPlace(self, value_left, value_right):
        for idx in range(len(value_left)):
             value_left[idx] += value_right[idx]
        return value_left

# Первый аргумент - начальные значения переменной
# Второй аргумент - Объект созданного пользовательского класса аккумулятора

vector_acc = sc.accumulator([1.0, 2.0, 3.0], VectorAccumulatorParam())
vector_acc.value

[1.0, 2.0, 3.0]

In [12]:
def vector_add(x):
    global vector_acc
    vector_acc += [x] * 3
    
rdd = sc.parallelize([1, 2, 3])
rdd.foreach(vector_add)
vector_acc.value

[7.0, 8.0, 9.0]

Концепция accumulator переменных не ограничивается только математической операцией сложения. Можно реализовывать свою произвольную логику аккумуляции данных.

Например, как ниже реализована аккумуляция строк через механизм их конкатенации.

In [13]:
class StringAccumulator(AccumulatorParam):
    def zero(self, value):
        return value
    def addInPlace(self, s1, s2):
        s1 += s2
        return s1

string_acc = sc.accumulator("", StringAccumulator())
string_acc.value

''

In [14]:
rdd = sc.parallelize(["S", "p", "a", "r", "k"])
rdd.foreach(lambda x: string_acc.add(x))
string_acc.value

'arkSp'

Так как операция конкатенации строк не является коммутативной:  $x + y\neq y + x$, то данный тип аккумулятора может выдавать разные результаты операций при повторных запусках, так как неизвестно какой из woker'ов первым произведёт сложение своей порции данных

In [15]:
string_acc.value = ""

rdd = sc.parallelize(["S", "p", "a", "r", "k"])
rdd.foreach(lambda x: string_acc.add(x))
string_acc.value

'arkSp'

#### `Broadcast`

![broadcast](images/broadcast.png)

Дополнением к WO (Write Only) переменным (аккумуляторам) являются RO (Read Only) переменные.
В Spark Read-only переменные называюся broadcast-переменными.
Broadcast позволяет отправить на каждый воркер копию данных, которые затем можно использовать локально. Данная копия данных отправляется на каждый из worker-ов в момент её инициализации. 

In [17]:
states = {"NY": "New York", "CA": "California", "FL": "Florida"}
broadcast_states = sc.broadcast(states)
broadcast_states.value

{'NY': 'New York', 'CA': 'California', 'FL': 'Florida'}

Если во многих оперциях используются одни и те же значения заранее определённые значения данных, то имеет смысл данные значения поместить в broadcast-переменную и отправить на каждый из worker-нод. Таким образом уменьшив сетевые взаимодействия при обращении к этим данным.

Типичным примером использования broadcast-переменных является применение их для отображения некоторых значений (lookup) как в примере  ниже

In [18]:
data = [
    ("James", "Smith", "USA", "CA"),
    ("Michael", "Rose", "USA", "NY"),
    ("Robert", "Williams", "USA", "CA"),
    ("Maria", "Jones", "USA", "FL")
]

rdd_03 = sc.parallelize(data)

def state_convert(code):
    return broadcast_states.value[code]

result = rdd_03.map(
    lambda x: (x[0], x[1], x[2], state_convert(x[3]))
).collect()
result

[('James', 'Smith', 'USA', 'California'),
 ('Michael', 'Rose', 'USA', 'New York'),
 ('Robert', 'Williams', 'USA', 'California'),
 ('Maria', 'Jones', 'USA', 'Florida')]

Также можно использовать broadcast-переменные в задаче фильтрации данных, если количество данных, по которым нужно производить её невелико.

In [19]:
filtered_states = sc.broadcast(["NY", "FL"])

filtered = rdd_03.filter(
    lambda x: x[3] in filtered_states.value
).collect()
filtered

[('Michael', 'Rose', 'USA', 'NY'), ('Maria', 'Jones', 'USA', 'FL')]

#### `Broadcast JOIN`

И ещё одним распространённым вариантом использования Broadcast является объединение таблиц, одна из которых "маленького" размера. В таком случае может оказаться выгоднее отправить копию меньшей таблицы на каждый воркер и выполнить Join локально, нежели чем выполнять распределённое объединение таблиц через обмен данными по сети.

Нужно учитывать, что пересылка больших таблиц по сети может оказаться дорогостоящей, поэтому выбор между Broadcast Join и "обычным" Join зависит от конкретной конфигурации кластера.

In [20]:
# Можно задать размер DataFrame, при котором join будет автоматически происходить через broadcast этой таблицы
# Размер задаётся в байтах. В данном случае — 100Мб.
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 104857600)

# Значение -1 отключает Broadcast Join
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

Скачаем [m5-forecast датасет](https://www.kaggle.com/c/m5-forecasting-accuracy), как это было сделано в предыдущих лекциях

In [21]:
import json
import requests
import subprocess
import zipfile

folder_url = 'https://disk.yandex.lt/d/JnDy1h48pJI7IA'
file_url = '/m5-forecating-accuracy.zip'
# запрос ссылки на скачивание
response = requests.get('https://cloud-api.yandex.net/v1/disk/public/resources/download',
                 params={'public_key': folder_url, 'path': file_url}) 
# 'парсинг' ссылки на скачивание
data_link = response.json()['href'] 	

filename = 'm5-forecating-accuracy.zip'
path = "./m5-forecasting-accuracy"

# запускаем скачивание вызовом команды wget из python
subprocess.run(
    ['wget', '-O', filename, data_link], # команда для исполнения
    stdout=subprocess.DEVNULL, # убираем печать отладочной информации
    stderr=subprocess.STDOUT
)

#распакуем данные по пути path
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall(path)

In [39]:
# Зададим пути к файлам из датасета
file_calendar = f"{path}/calendar.csv"
file_validation = f"{path}/sales_train_validation.csv"
file_evaluation = f"{path}/sales_train_evaluation.csv"
file_prices = f"{path}/sell_prices.csv"
file_calendar = f"{path}/calendar.csv"

file_type = "csv"
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

df_validation = (
    spark.read.format(file_type)
      .option("inferSchema", infer_schema)
      .option("header", first_row_is_header)
      .option("sep", delimiter)
      .load(file_validation)
)
df_evaluation = (
    spark.read.format(file_type)
      .option("inferSchema", infer_schema)
      .option("header", first_row_is_header)
      .option("sep", delimiter)
      .load(file_evaluation)
)
df_prices = (
    spark.read.format(file_type)
      .option("inferSchema", infer_schema)
      .option("header", first_row_is_header)
      .option("sep", delimiter)
      .load(file_prices)
)
df_calendar = (
    spark.read.format(file_type)
      .option("inferSchema", infer_schema)
      .option("header", first_row_is_header)
      .option("sep", delimiter)
      .load(file_calendar)
)

In [40]:
df_evaluation.limit(5).toPandas()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


Так как ML-алгоритмы не работают с текстовыми данными, зачастую приходится преобразовывать текстовое предстваление данных в численное. Например, в датафрейме выше такими признаками являются колонки: cat_id, state_id и store_id.

In [41]:
df_evaluation.select(
    df_evaluation.cat_id
).distinct().show()

+---------+
|   cat_id|
+---------+
|    FOODS|
|HOUSEHOLD|
|  HOBBIES|
+---------+



Создадим таблицу-отображение: текст -> число

In [42]:
cat_id_hex =[
    ('FOODS', 0),
    ('HOUSEHOLD', 2),
    ('HOBBIES', 3)
]
small_df = spark.createDataFrame(data=cat_id_hex, schema=['cat_id', 'code'])
small_df.show()

+---------+----+
|   cat_id|code|
+---------+----+
|    FOODS|   0|
|HOUSEHOLD|   2|
|  HOBBIES|   3|
+---------+----+



Для того, чтобы пометить, что некоторый Spark DataFrame должен быть отправлен на все worker-ноды, необходимо вопспользовтаться функцией `broadcast`.

In [43]:
broadcast_join_df = df_evaluation.join(
  F.broadcast(small_df), small_df.cat_id == df_evaluation.cat_id
)
broadcast_join_df.limit(1).toPandas()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,cat_id,code
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,3,3,0,1,HOBBIES,3


Теперь сравним количество операций между join с broadcast-dataframe и просто DataFrame, у которого данные разбиты по worker-нодам

In [44]:
broadcast_join_df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [cat_id#39091], [cat_id#45007], Inner, BuildRight, false
   :- Filter isnotnull(cat_id#39091)
   :  +- FileScan csv [id#39088,item_id#39089,dept_id#39090,cat_id#39091,store_id#39092,state_id#39093,d_1#39094,d_2#39095,d_3#39096,d_4#39097,d_5#39098,d_6#39099,d_7#39100,d_8#39101,d_9#39102,d_10#39103,d_11#39104,d_12#39105,d_13#39106,d_14#39107,d_15#39108,d_16#39109,d_17#39110,d_18#39111,... 1923 more fields] Batched: false, DataFilters: [isnotnull(cat_id#39091)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/evgeniy/github/spark_hse_dpo_2023/Lecture5/m5-forecasting-a..., PartitionFilters: [], PushedFilters: [IsNotNull(cat_id)], ReadSchema: struct<id:string,item_id:string,dept_id:string,cat_id:string,store_id:string,state_id:string,d_1:...
   +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, string, false]),false), [plan_id=685]
      +- Filter isnotnull(cat_id#45007)
         +- Scan Ex

In [45]:
join_df = df_evaluation.join(
  small_df, small_df.cat_id == df_evaluation.cat_id
)
join_df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [cat_id#39091], [cat_id#45007], Inner
   :- Sort [cat_id#39091 ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(cat_id#39091, 200), ENSURE_REQUIREMENTS, [plan_id=709]
   :     +- Filter isnotnull(cat_id#39091)
   :        +- FileScan csv [id#39088,item_id#39089,dept_id#39090,cat_id#39091,store_id#39092,state_id#39093,d_1#39094,d_2#39095,d_3#39096,d_4#39097,d_5#39098,d_6#39099,d_7#39100,d_8#39101,d_9#39102,d_10#39103,d_11#39104,d_12#39105,d_13#39106,d_14#39107,d_15#39108,d_16#39109,d_17#39110,d_18#39111,... 1923 more fields] Batched: false, DataFilters: [isnotnull(cat_id#39091)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/evgeniy/github/spark_hse_dpo_2023/Lecture5/m5-forecasting-a..., PartitionFilters: [], PushedFilters: [IsNotNull(cat_id)], ReadSchema: struct<id:string,item_id:string,dept_id:string,cat_id:string,store_id:string,state_id:string,d_1:...
   +- Sort [cat_id#45007 ASC NUL

Как видно при join'е с broadcast перменной количество необходимых операций меньше

### `Spark RDD Gradient Descent`

$$
D = \{(x_{i}, y_{i}) | x_{i} \in \mathbb{R}^{d}, y \in \mathbb{R}\}_{1}^{n}
$$
$$
{y}^{pred}_{i} = \langle x, w \rangle + b
$$
$$
L_{i} = \frac{1}{2} ({y}^{pred}_{i} - y_{i})^{2}
$$
$$
\mathfrak{L}(w, b) = \frac{1}{n}\sum\limits_{i=1}^{n} L_{i} \longrightarrow \min_{w, b}
$$

Необходимо найти оптимальные $w \in \mathbb{R}^{d}, b \in \mathbb{R}$

Один из вариантов решения задачи: Градиентный Спуск (GD):

$$
\space   w^{i+1} = w^{i} - \alpha \nabla_{w}\mathfrak{L}
$$
$$
\space  b^{i+1} = b^{i} - \alpha \nabla_{b}\mathfrak{L}
$$

При использовании линейной регрессии делается допущение о распределении данных:
$$
y_{i} \sim \mathcal{N}(\langle x_{i}, w^{*} \rangle, \sigma^{2})
$$

$$\nabla_{w} L = (\frac{\partial L}{\partial w_{1}}, ..., \frac{\partial L}{\partial w_{d}})$$


$$
\nabla_{w} \mathfrak{L} = \frac{1}{2n} \sum\limits_{i=1}^{n} \nabla_{w}({y}^{pred}_{i} - y_{i})({y}^{pred}_{i} - y_{i}) = \frac{1}{n} \sum\limits_{i=1}^{n} ({y}^{pred}_{i} - y_{i}) \nabla_{w}{y}^{pred}_{i} = \{y_{i}^{pred} = \langle x_{i}, w \rangle + b \} = \frac{1}{n} \sum\limits_{i=1}^{n} ({y}^{pred}_{i} - y_{i}) x_{i} = \frac{1}{n} \sum\limits_{i=1}^{n} (\langle x_{i}, w \rangle + b - y_{i}) x_{i}
$$

Сгенерируем модельные данные, на которых провверим работу алгоритма градиентного спуска

In [81]:
X = np.concatenate((np.random.randn(1000, 10), np.ones((1000, 1))), axis=1)
w_star = np.random.randn(X.shape[1])

y = X.dot(w_star) + 0.001 * np.random.randn(X.shape[0])

Перепишем с математической формулировки на ...

In [90]:
def gradient_descent(X, y, alpha=0.1, epochs=1):
    # 1. Иницализируем начальные значения весов
    # 2. Итеративно с кол-вом эпох N:
    #     a. Вычисляем ошибку, считаем по ней градиент
    #     b. Обновляем веса 
    # 3. На выходе из функции посчитанные веса w
    
    X_rdd = sc.parallelize(X).cache()
    y_rdd = sc.parallelize(y).cache()
    
    n = X_rdd.count()
    d = X_rdd.take(1)[0].shape[0]
    
    # Кэшируем результат вычислений, чтобы не перевычислять его на каждой итерации
    X_y_rdd = X_rdd.zip(y_rdd).cache()
    # инициализируем w нулевым вектором размерности d
    w = np.zeros(d)
    for epoch in range(epochs):
        # на каждой эпохе инициализируем общее значение весов
        # для всех worker-нод и размерность n
        w_br = sc.broadcast(w)
        n_br = sc.broadcast(n)
        
        total_error = sc.accumulator(0.0)
        def grad_mapper(x_y, total_error):
            delta = (np.sum(x_y[0] * w_br.value) - x_y[1])
            error = (delta ** 2.0) / 2.0
            total_error.add(error / n_br.value)
            return x_y[0] * delta
        
        grad = X_y_rdd.map(lambda x: grad_mapper(x, total_error=total_error)).sum() / n
        w = w - alpha * grad
        
        if epoch % 10 == 0:
            print('Epoch: {0:d}, Loss {1:.3f}'.format(epoch, total_error.value / n))
    return w

gradient_descent(X, y, alpha=0.05, epochs=50)

In [92]:
w_star

array([ 1.07914287, -0.53380276,  2.03264089,  0.54738543,  1.30964064,
       -0.69661485, -0.78671941,  1.17804389,  0.6113177 , -0.91477037,
       -0.01177458])

### `Spark Winsorizing`

Винзоризация — это метод предобработки численных данных, при котором значения за пределами заданных квантилей заменяются на значения этих квантилей. 

![quantiles](q.png)

Например, $X = \{100, 6, 52, 26, 8, 81, 52, 15, 2, 74, 93, 82, 36, 22, 74, 90, 97, 50, 4, 40, 1\}$.

Винзоризация для $0.1$ и $0.9$ квантилей выполняется следующим образом:
1. Определяем квантили: $q_{0.1} = 3, q_{0.9} = 93$
2. Заменяем все значения меньше $3$ на $3$ и больше $93$ на $93$: 

$$\hat{X} = \{93, 6, 52, 26, 8, 81, 52, 15, 3, 74, 93, 82, 36, 22, 74, 90, 97, 50, 4, 40, 3\}$$

Такая предобработка убирает экстремальные значения и выбросы, что приводит к более надёжному посчёту статистик по выборке (матожидание, дисперисия и так далее).

In [26]:
def winsorizing(
    df: pyspark.sql.dataframe.DataFrame, 
    column: str = 'sales',
    lower_percentile: float = 0.1,
    higher_percentile: float = 0.9
) -> pyspark.sql.dataframe.DataFrame:
    
    wspec = Window().partitionBy()
    
    lp_column = '_'.join([column, 'lower_percentile'])
    hp_column = '_'.join([column, 'higher_percentile'])
    df = df.withColumns({
        lp_column: F.percentile_approx(F.col(column), lower_percentile).over(wspec),
        hp_column: F.percentile_approx(F.col(column), higher_percentile).over(wspec)
    })
    
    df = (
        df
            .withColumn(
                '_'.join([column, 'winsorized']),
                
                F.when(
                    # если значение меньше левой квантили - заменяем
                    F.col(column) < F.col(lp_column),
                    F.col(lp_column)
                ).otherwise(
                    F.when(
                        # если значение больше правой квантили - заменяем на неё
                        F.col(column) > F.col(hp_column),
                        F.col(hp_column)
                    ).otherwise(
                        # иначе 
                        F.col(column)
                    )
                )
            )
    )
    
    return df

In [27]:
data = [
    (0.77347701,  ),
    (0.77617723, ),
    (-0.26191574,  ),
    (0.06015559, ),
    (-0.18058041,),
    (1.15605904, ),
    (-0.54163328,  ),
    (0.83280377,),
    (-0.69920523, ),
    (-0.33986035,),
    (-0.94114708, ),
    (-0.88438698,  ),
    (1.18682329,  ),
    (1.21287342, ),
    (-0.82575258,),
    (0.5895868, ),
    (-1.646899, ),
    (-1.5341987, ),
    (-0.94135006,  ),
    (0.5699716,)
]
df = spark.createDataFrame(data, ['sales'])
df.show()

+-----------+
|      sales|
+-----------+
| 0.77347701|
| 0.77617723|
|-0.26191574|
| 0.06015559|
|-0.18058041|
| 1.15605904|
|-0.54163328|
| 0.83280377|
|-0.69920523|
|-0.33986035|
|-0.94114708|
|-0.88438698|
| 1.18682329|
| 1.21287342|
|-0.82575258|
|  0.5895868|
|  -1.646899|
| -1.5341987|
|-0.94135006|
|  0.5699716|
+-----------+



In [28]:
winsorizing(df).show()

23/02/21 11:34:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/21 11:34:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/21 11:34:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/21 11:34:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/21 11:34:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+-----------+----------------------+-----------------------+----------------+
|      sales|sales_lower_percentile|sales_higher_percentile|sales_winsorized|
+-----------+------

### `Нормализация данных`

##### `Standard Scaler`

Отнормализуем данные, так чтобы их среднее было нулевым, а стандартное отклонение единичным.

$$ X = \frac{X-\mu}{\sigma}$$

- $\mu = EX$ - среднее значение признака, посчитанного по всем данным

- $\sigma = \sqrt{DX} = \sqrt{E[X - EX]^{2}}$ - его стадартное нормальное отклонение

![normalization](images/normalization.jpeg)

In [199]:
# сгенерируем модельные данные из равномерного распредления на отрезку [0, 20]

data = [
    (random.uniform(0, 20), )
    for i in range(20)
]

df = spark.createDataFrame(data, ['sales'])
df.show()

+------------------+
|             sales|
+------------------+
| 4.142494366086895|
| 9.639546895246765|
| 18.25011321949261|
|15.352734159760217|
|19.405881111477253|
| 10.28101061161016|
|13.575668268235043|
|16.256219883800973|
|14.069543501302437|
|11.595794759712811|
|11.025936207941449|
|3.0266474902194096|
| 8.129449547044569|
| 7.974603647706955|
|3.9665375554392357|
| 9.840458658422351|
|1.8000164983523548|
|5.7111954986306195|
|1.1489917593161625|
|19.517643212735482|
+------------------+



In [203]:
def normalize(df, input_column, output_column):
    
    wspec = Window().partitionBy()
    
    df = df.withColumn("mu", F.mean(input_column).over(wspec))
    df = df.withColumn("sigma", F.sqrt(F.mean((F.col(input_column) - F.col("mu")) ** 2).over(wspec)))

    df = df.withColumn(output_column, (F.col(input_column) - F.col("mu")) / F.col("sigma"))
    
    return df

In [204]:
normalized_df = normalize(df, "sales", "normalized_sales")
normalized_df.show()

23/11/11 08:11:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 08:11:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 08:11:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 08:11:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 08:11:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------------------+------------------+-----------------+--------------------+
|             sales|                mu|            sigma|    normalized_sales|
+------------------+------------------+-----------------+--------------------+
| 4.142494366086895|10.235524342626688|5.647561211979469| -1.0788780763660262|
| 9.639546895246765|10.235524342626688|5.647561211979469|-0.10552828468963736|
| 18.25011321949261|10.235524342626688|5.647561211979469|  1.4191238617946405|
|15.352734159760217|10.235524342626688|5.647561211979469|  0.9060919616557725|
|19.405881111477253|10.235524342626688|5.647561211979469|  1.6237728861439569|
| 10.28101061161016|10.235524342626688|5.647561211979469|0.008054143598654218|
|13.575668268235043|10.235524342626688|5.647561211979469|  0.5914312037067122|
|16.256219883800973|10.235524342626688|5.647561211979469|  1.0660699929030133|
|14.069543501302437|10.235524342626688|5.647561211979469|  0.6788804963358558|
|11.595794759712811|10.235524342626688|5.64756121197

23/11/11 08:11:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 08:11:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 08:11:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 08:11:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [206]:
# посмотрим чему теперь равны среднее и стандартное отклонение для нормализоавнного датафрейма
normalize(normalized_df, "normalized_sales", "tmp").show(20, False)

23/11/11 08:12:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 08:12:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 08:12:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 08:12:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 08:12:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 08:12:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 0

+------------------+-----------------------+------------------+--------------------+--------------------+
|sales             |mu                     |sigma             |normalized_sales    |tmp                 |
+------------------+-----------------------+------------------+--------------------+--------------------+
|4.142494366086895 |-1.1102230246251566E-17|0.9999999999999999|-1.0788780763660262 |-1.0788780763660264 |
|9.639546895246765 |-1.1102230246251566E-17|0.9999999999999999|-0.10552828468963736|-0.10552828468963736|
|18.25011321949261 |-1.1102230246251566E-17|0.9999999999999999|1.4191238617946405  |1.4191238617946407  |
|15.352734159760217|-1.1102230246251566E-17|0.9999999999999999|0.9060919616557725  |0.9060919616557727  |
|19.405881111477253|-1.1102230246251566E-17|0.9999999999999999|1.6237728861439569  |1.623772886143957   |
|10.28101061161016 |-1.1102230246251566E-17|0.9999999999999999|0.008054143598654218|0.00805414359865423 |
|13.575668268235043|-1.1102230246251566E-17|0.

23/11/11 08:12:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 08:12:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 08:12:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 08:12:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 08:12:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 08:12:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 0

##### `MinMax Scaler`

Если необходимо преобразовать данные к заранее заданному интервалу, например, к единичному интервалу [0, 1], то тут подойдёт min-max нормализация.

$$X = \frac{X - min(X)}{max(X) - min(X)}$$

In [214]:
def min_max_normalize(df, input_column, output_column):
    
    wspec = Window().partitionBy()
    
    df = df.withColumn("max", F.max(input_column).over(wspec))
    df = df.withColumn("min", F.min(input_column).over(wspec))

    df = df.withColumn(output_column, (F.col(input_column) - F.col("min")) / (F.col("max") - F.col("min")))
    
    return df

In [215]:
normalized_df = min_max_normalize(df, "sales", "normalized_sales")
normalized_df.show()

23/11/11 08:17:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 08:17:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 08:17:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------------------+------------------+------------------+-------------------+
|             sales|               max|               min|   normalized_sales|
+------------------+------------------+------------------+-------------------+
| 4.142494366086895|19.517643212735482|1.1489917593161625| 0.1629680117978118|
| 9.639546895246765|19.517643212735482|1.1489917593161625| 0.4622307281218561|
| 18.25011321949261|19.517643212735482|1.1489917593161625| 0.9309949346876568|
|15.352734159760217|19.517643212735482|1.1489917593161625| 0.7732599443384851|
|19.405881111477253|19.517643212735482|1.1489917593161625| 0.9939156066224216|
| 10.28101061161016|19.517643212735482|1.1489917593161625| 0.4971523835297161|
|13.575668268235043|19.517643212735482|1.1489917593161625| 0.6765154502730607|
|16.256219883800973|19.517643212735482|1.1489917593161625|  0.822446229261572|
|14.069543501302437|19.517643212735482|1.1489917593161625| 0.7034023033618573|
|11.595794759712811|19.517643212735482|1.14899175931

23/11/11 08:17:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/11 08:17:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
